# Assignment 5 - Kanak Agarwal

### Problem 1

In [ ]:
import numpy as np

n = 10000
f_true = (np.pi**4)/90

# Forward sum
f_forward = np.float32(0.0)
for i in range(1, n+1):
    f_forward += np.float32(1.0 / (i**4))

# Reverse sum
f_reverse = np.float32(0.0)
for i in range(n, 0, -1):
    f_reverse += np.float32(1.0 / (i**4))

err_forward = abs((f_forward - f_true)/f_true) * 100
err_reverse = abs((f_reverse - f_true)/f_true) * 100

print(f"Forward sum = {f_forward}, Error = {err_forward:.6f}%")
print(f"Reverse sum = {f_reverse}, Error = {err_reverse:.6f}%")

Forward sum = 1.082322120666504, Error = 0.000099%
Reverse sum = 1.0823231935501099, Error = 0.000000%


In [6]:
import numpy as np
import sys

M, N, P = 200, 400, 520
array = np.zeros((M, N, P), dtype=np.float64) # float64 - double precision

num_elements = array.size
memory_bytes = array.nbytes
memory_GB = memory_bytes / (2**30)

print(f"Array dimensions: {M} x {N} x {P}")
print(f"Number of elements = {num_elements:,}")
print(f"Each element size = {array.itemsize} bytes")
print(f"Total size = {memory_bytes:,} bytes ({memory_GB:.2f} GB)")

print(f"System-reported size (including Python object overhead): {sys.getsizeof(array):,} bytes")


Array dimensions: 200 x 400 x 520
Number of elements = 41,600,000
Each element size = 8 bytes
Total size = 332,800,000 bytes (0.31 GB)
System-reported size (including Python object overhead): 332,800,144 bytes


## Problem 2

$$ y = -x^2 + x + 0.75 $$
$$ y + 5xy = x^2 $$

Solve using fixed point and Newton Raphson methods for x = y = 1.2.

In [93]:
import math

def fixed_point(x0, y0, tol=1e-10, max_iter=200):
    x, y = x0, y0
    history = [(0, x, y)]
    for k in range(1, max_iter+1):
        y_new = -x**2 + x + 0.75
        disc = 25*y_new*y_new + 4*y_new
        if disc < 0:
            raise ValueError("Negative discriminant in fixed-point step")
        r1 = (5*y_new + math.sqrt(disc)) / 2.0
        r2 = (5*y_new - math.sqrt(disc)) / 2.0
        x_new = r1 if abs(r1 - x) < abs(r2 - x) else r2
        history.append((k, x_new, y_new))

        if abs(x_new - x) < tol and abs(y_new - y) < tol:
            return x_new, y_new, k, history
        x, y = x_new, y_new

    return x, y, max_iter, history

x0 = 1.2; y0 = 1.2
xf, yf, iters_fp, hist_fp = fixed_point(x0, y0)
for row in hist_fp:
    print("Iter {:3d}: x = {:.10f}, y = {:.10f}".format(row[0], row[1], row[2]))


Iter   0: x = 1.2000000000, y = 1.2000000000
Iter   1: x = -0.1863777746, y = 0.5100000000
Iter   2: x = -0.1868040445, y = 0.5288855506
Iter   3: x = -0.1867912353, y = 0.5283002045
Iter   4: x = -0.1867916206, y = 0.5283177992
Iter   5: x = -0.1867916091, y = 0.5283172698
Iter   6: x = -0.1867916094, y = 0.5283172857
Iter   7: x = -0.1867916094, y = 0.5283172853
Iter   8: x = -0.1867916094, y = 0.5283172853


In [95]:
import numpy as np

def f_J(x, y):
    f1 = y + x*x - x - 0.75
    f2 = y + 5*x*y - x*x
    J = np.array([[2*x - 1, 1.0],
                  [-2*x + 5*y, 1.0 + 5*x]], dtype=float)
    F = np.array([f1, f2], dtype=float)
    return F, J

def newton_raphson_multi(x0, y0, tol=1e-10, max_iter=50):
    x, y = x0, y0
    history = [(0, x, y)]
    for k in range(1, max_iter+1):
        F, J = f_J(x, y)
        delta = np.linalg.solve(J, -F)
        x_new = x + delta[0]
        y_new = y + delta[1]
        history.append((k, x_new, y_new, np.linalg.norm(delta)))
        if np.linalg.norm(delta) < tol:
            return x_new, y_new, k, history
        x, y = x_new, y_new
    return x, y, max_iter, history

x0 = 1.2; y0 = 1.2
xn, yn, iters_nr, hist_nr = newton_raphson_multi(x0, y0)
for row in hist_nr:
    print("Iter {:3d}: x = {:.10f}, y = {:.10f}".format(row[0], row[1], row[2]))


Iter   0: x = 1.2000000000, y = 1.2000000000
Iter   1: x = 1.5435483871, y = 0.0290322581
Iter   2: x = 1.3941229465, y = 0.2228721189
Iter   3: x = 1.3724545855, y = 0.2392925142
Iter   4: x = 1.3720655204, y = 0.2395018794
Iter   5: x = 1.3720654058, y = 0.2395019280
Iter   6: x = 1.3720654058, y = 0.2395019280


## Problem 4

In [13]:
import math

R = 3.0
V_target = 30.0
pi = math.pi

def V(h):
    return (1/3) * pi * h**2 * (3*R - h)

def f(h):
    return V(h) - V_target

def df(h):
    return (1/3) * pi * (6*R*h - 3*h**2)

h = 3.0 # Initial guess
iter = []

for i in range(3):
    fh = f(h)
    dfh = df(h)
    delta = fh / dfh
    h_new = h - delta
    # approximate relative error after iteration: |(h_new - h)/h_new|
    approx_rel_err = abs((h_new - h)/h_new)
    iter.append({
        'iteration': i,
        'h_old': h,
        'f(h)': fh,
        "f'(h)": dfh,
        'delta': delta,
        'h_new': h_new,
        'approx_rel_err': approx_rel_err
    })
    h = h_new

# Results
print(f"{'Iter':<5} {'h_old':<12} {'f(h)':<15} {'f\'(h)':<15} {'delta':<15} {'h_new':<12} {'Approx Rel Err':<15}")
for entry in iter:
    print(f"{entry['iteration']:<5} {entry['h_old']:<12.6f} {entry['f(h)']:<15.6f} {entry['f\'(h)']:<15.6f} {entry['delta']:<15.6f} {entry['h_new']:<12.6f} {entry['approx_rel_err']:<15.6e}")  

Iter  h_old        f(h)            f'(h)           delta           h_new        Approx Rel Err 
0     3.000000     26.548668       28.274334       0.938967        2.061033     4.555808e-01   
1     2.061033     0.866921        25.504520       0.033991        2.027042     1.676871e-02   
2     2.027042     0.003449        25.300354       0.000136        2.026906     6.726269e-05   


## Problem 5

$$ f(x) = 0.0074x^4 - 0.284x^3 + 3.355x^2 - 12.183x + 5 $$

In [20]:
def f(x):
    return 0.0074*x**4 - 0.284*x**3 + 3.355*x**2 - 12.183*x + 5.0

def df(x):
    return 4*0.0074*x**3 - 3*0.284*x**2 + 2*3.355*x - 12.183

x = 17 # Initial guess
tol = 1e-6
max_iter = 100

results = []
for i in range(1, max_iter+1):
    fx = f(x)
    dfx = df(x)
    if dfx == 0:
        results.append((i, x, fx, dfx, None, "Zero derivative"))
        break
    delta = fx/dfx
    x_new = x - delta
    approx_rel_err = abs((x_new - x)/x_new) if x_new != 0 else abs(x_new - x)
    results.append((i, x, fx, dfx, delta, x_new, approx_rel_err))
    x = x_new
    if abs(delta) < tol:
        break

print(f"{'Iter':<5} {'x_old':<12} {'f(x)':<15} {'f\'(x)':<15} {'delta':<15} {'x_new':<12} {'Approx Rel Err':<15}")
for res in results:
    if len(res) == 6:
        iter_num, x_old, fx, dfx, delta, message = res
        print(f"{iter_num:<5} {x_old:<12.6f} {fx:<15.6f} {dfx:<15.6f} {'N/A':<15} {'N/A':<12} {message:<15}")
    else:
        iter_num, x_old, fx, dfx, delta, x_new, approx_rel_err = res
        print(f"{iter_num:<5} {x_old:<12.6f} {fx:<15.6f} {dfx:<15.6f} {delta:<15.6f} {x_new:<12.6f} {approx_rel_err:<15.6e}")

f15 = f(15)
f20 = f(20)
print()
print(f"f(15) = {f15:.6f}")
print(f"f(20) = {f20:.6f}")


Iter  x_old        f(x)            f'(x)           delta           x_new        Approx Rel Err 
1     17.000000    -9.752600       1.083800        -8.998524       25.998524    3.461167e-01   
2     25.998524    346.103090      106.541483      3.248529        22.749995    1.427925e-01   
3     22.749995    102.532341      48.032765       2.134633        20.615361    1.035458e-01   
4     20.615361    28.042877       23.388984       1.198978        19.416383    6.175084e-02   
5     19.416383    6.152592        13.569363       0.453418        18.962966    2.391071e-02   
6     18.962966    0.703226        10.525864       0.066809        18.896156    3.535604e-03   
7     18.896156    0.014042        10.106728       0.001389        18.894767    7.353361e-05   
8     18.894767    0.000006        10.098090       0.000001        18.894766    3.145300e-08   

f(15) = -6.745000
f(20) = 15.340000


## Problem 6

In [ ]:
import math

def y_plus(x):
    arg = 16 - (x+1)**2
    if arg < 0: return None  # outside domain
    return 2.0 + math.sqrt(arg)

def y_minus(x):
    arg = 16 - (x+1)**2
    if arg < 0: return None
    return 2.0 - math.sqrt(arg)

def secant_method(func, x0, x1, tol=1e-6, max_iter=100):
    results = []
    for i in range(1, max_iter+1):
        f_x0 = func(x0)
        f_x1 = func(x1)
        if f_x0 is None or f_x1 is None:
            results.append((i, x0, f_x0, x1, f_x1, None, "Out of domain"))
            break
        if f_x1 - f_x0 == 0:
            results.append((i, x0, f_x0, x1, f_x1, None, "Zero denominator"))
            break
        x2 = x1 - f_x1 * (x1 - x0) / (f_x1 - f_x0)
        approx_rel_err = abs((x2 - x1)/x2) if x2 != 0 else abs(x2 - x1)
        results.append((i, x0, f_x0, x1, f_x1, x2, approx_rel_err))
        if abs(x2 - x1) < tol:
            break
        x0, x1 = x1, x2
    return results

# Initial guesses
x0, x1 = 0.5, 3.0

results_plus = secant_method(y_plus, x0, x1)
results_minus = secant_method(y_minus, x0, x1)

print("Secant Method for y_plus:")
print(f"{'Iter':<5} {'x0':<12} {'f(x0)':<22} {'x1':<12} {'f(x1)':<25} {'x2':<20} {'Approx Rel Err':<15}")
for res in results_plus:
    print(f"{res[0]:<5} {res[1]:<12.6f} {res[2] if res[2] is not None else 'N/A':<22} {res[3]:<12.6f} {res[4] if res[4] is not None else 'N/A':<25} {res[5] if res[5] is not None else 'N/A':<20} {res[6] if res[6] is not None else 'N/A':<20}")

print("\nSecant Method for y_minus:")
print(f"{'Iter':<5} {'x0':<12} {'f(x0)':<22} {'x1':<12} {'f(x1)':<25} {'x2':<20} {'Approx Rel Err':<15}")
for res in results_minus:
    print(f"{res[0]:<5} {res[1]:<12.6f} {res[2] if res[2] is not None else 'N/A':<22} {res[3]:<12.6f} {res[4] if res[4] is not None else 'N/A':<25} {res[5] if res[5] is not None else 'N/A':<20} {res[6] if res[6] is not None else 'N/A':<20}")

Secant Method for y_plus:
Iter  x0           f(x0)                  x1           f(x1)                     x2                   Approx Rel Err 
1     0.500000     5.7080992435478315     3.000000     2.0                       4.348399724926484    0.3100910243363794  
2     3.000000     2.0                    4.348400     N/A                       N/A                  Out of domain       

Secant Method for y_minus:
Iter  x0           f(x0)                  x1           f(x1)                     x2                   Approx Rel Err 
1     0.500000     -1.7080992435478315    3.000000     2.0                       1.651600275073516    0.81642013826043    
2     3.000000     2.0                    1.651600     -0.9948315447166731       2.099515478721826    0.21334217736798897 
3     1.651600     -0.9948315447166731    2.099515     -0.5284390040425753       2.607019015912479    0.19466813785898776 
4     2.099515     -0.5284390040425753    2.607019     0.2710078603863548        2.434978253890

## Problem 7

$$ G(s) = \frac{C(s)}{N(s)} = \frac{s^3 + 12.5s^2 + 50.5s + 66}{s^4 + 19s^3 + 122s^2 + 296s + 192} $$

In [73]:
import numpy as np

def poly_eval(coeffs, x): # Evaluate the polynomial and its derivative at x using Horner's method
    n = len(coeffs) - 1
    p, dp = coeffs[0], 0
    for i in range(1, n + 1):
        dp = dp * x + p
        p = p * x + coeffs[i]
    return p, dp

def newton_root(coeffs, x0, tol=1e-8, max_iter=100): # Find the root using Newton-Raphson
    x = x0
    for _ in range(max_iter):
        fx, dfx = poly_eval(coeffs, x)
        if abs(dfx) < 1e-12:
            break
        x_new = x - fx / dfx
        if abs(x_new - x) < tol:
            return x_new
        x = x_new
    return x

def simplify(coeffs, root):  # divide the polynomial by (x - root)
    n = len(coeffs)
    new_coeffs = [coeffs[0]]
    for i in range(1, n - 1):
        new_coeffs.append(coeffs[i] + root * new_coeffs[-1])
    remainder = coeffs[-1] + root * new_coeffs[-1]
    return np.array(new_coeffs, dtype=float), remainder

def recursive_roots(coeffs, guesses):
    roots = []
    c = coeffs.copy()
    for g in guesses:
        root = newton_root(c, g)
        roots.append(root)
        c, _ = simplify(c, root)
    return roots

num = [1, 12.5, 50.5, 66]
den = [1, 19, 122, 296, 192]

guesses = [-3, -4, -5]
roots_num = recursive_roots(num, guesses)

guesses = [-1, -3, -5, -7]
roots_den = recursive_roots(den, guesses)

print("Numerator Roots:")
for r in roots_num:
    print(f"{r:.6f}")

print("\nDenominator Roots:")
for r in roots_den:
    print(f"{r:.6f}")



Numerator Roots:
-3.000000
-4.000000
-5.500000

Denominator Roots:
-1.000000
-4.000000
-6.000000
-8.000000


## Problem 8

In [ ]:
import numpy as np

def poly_eval(coeffs, x): # Evaluate the polynomial and its derivative at x using Horner's method
    n = len(coeffs) - 1
    p, dp = coeffs[0], 0
    for i in range(1, n + 1):
        dp = dp * x + p
        p = p * x + coeffs[i]
    return p, dp

def newton_root(coeffs, x0, tol=1e-8, max_iter=100): # Find the root using Newton-Raphson
    x = x0
    for _ in range(max_iter):
        fx, dfx = poly_eval(coeffs, x)
        if abs(dfx) < 1e-12:
            break
        x_new = x - fx / dfx
        if abs(x_new - x) < tol:
            return x_new
        x = x_new
    return x

def simplify(coeffs, root):  # divide the polynomial by (x - root)
    n = len(coeffs)
    new_coeffs = [coeffs[0]]
    for i in range(1, n - 1):
        new_coeffs.append(coeffs[i] + root * new_coeffs[-1])
    remainder = coeffs[-1] + root * new_coeffs[-1]
    return np.array(new_coeffs, dtype=float), remainder

def recursive_roots(coeffs, guesses):
    roots = []
    c = coeffs.copy()
    for g in guesses:
        root = newton_root(c, g)
        roots.append(root)
        c, _ = simplify(c, root)
    return roots

poly = [1, -33, -704, -1859]
guesses = [50, -5, -10]
roots_num = recursive_roots(poly, guesses)

print("Roots:")
for r in roots_num:
    print(f"{r:.6f}")

Roots:
48.354284
-3.150211
-12.204073


## Problem 9

In [83]:
import math, numpy as np
# Newton's method for system of equations - multivariate problem

T0 = 450.0
T3 = 25.0

def q1(T1):
    return 1e-9 * ( (T0 + 273.0)**4 - (T1 + 273.0)**4 )

def dq1_dT1(T1):
    return -1e-9 * 4.0 * (T1 + 273.0)**3

def q2(T1, T2):
    return 4.0 * (T1 - T2)

def dq2_dT1(T1, T2):
    return 4.0
def dq2_dT2(T1, T2):
    return -4.0

def q3(T2):
    return 1.3 * (T2 - T3)**(4.0/3.0)

def dq3_dT2(T2):
    return 1.3 * (4.0/3.0) * (T2 - T3)**(1.0/3.0)

def residuals(T1, T2):
    r1 = q1(T1) - q2(T1, T2)
    r2 = q2(T1, T2) - q3(T2)
    return np.array([r1, r2], dtype=float)

def jacobian(T1, T2):
    # dr1/dT1, dr1/dT2; dr2/dT1, dr2/dT2
    dr1_dT1 = dq1_dT1(T1) - dq2_dT1(T1, T2)
    dr1_dT2 = - dq2_dT2(T1, T2)  # dq1/dT2 = 0
    dr2_dT1 = dq2_dT1(T1, T2)
    dr2_dT2 = dq2_dT2(T1, T2) - dq3_dT2(T2)
    return np.array([[dr1_dT1, dr1_dT2],[dr2_dT1, dr2_dT2]], dtype=float)

# Newton iterations
T1 = 200.0
T2 = 100.0
tol = 1e-10
max_iter = 50
history = []

for k in range(1, max_iter+1):
    F = residuals(T1, T2)
    J = jacobian(T1, T2)
    delta = np.linalg.solve(J, -F)
    T1_new = T1 + delta[0]
    T2_new = T2 + delta[1]
    approx_rel_err = np.max(np.abs(delta) / np.maximum([abs(T1_new), abs(T2_new)], 1e-12))
    history.append((k, T1, T2, F[0], F[1], delta[0], delta[1], T1_new, T2_new, approx_rel_err))
    T1, T2 = T1_new, T2_new
    if approx_rel_err < tol:
        break

# Results
print("iter     T1_old    T2_old    r1        r2        dT1        dT2    T1_new    T2_new   rel_err")
for row in history:
    print("{:3d} {:10.6f} {:10.6f} {:10.6e} {:10.6e} {:10.6e} {:10.6e} {:10.6f} {:10.6f} {:10.6e}".format(*row))

print("\nFinal T1, T2:", T1, T2)
print("Final fluxes q1, q2, q3:", q1(T1), q2(T1,T2), q3(T2))

iter     T1_old    T2_old    r1        r2        dT1        dT2    T1_new    T2_new   rel_err
  1 200.000000 100.000000 -1.768091e+02 -1.117342e+01 -6.008165e+01 -2.223745e+01 139.918353  77.762549 4.294050e-01
  2 139.918353  77.762549 -4.448395e+00 -8.640004e+00 -2.806216e+00 -1.891684e+00 137.112137  75.870865 2.493294e-02
  3 137.112137  75.870865 -8.019605e-03 -7.408143e-02 -1.329950e-02 -1.221197e-02 137.098838  75.858653 1.609832e-04
  4 137.098838  75.858653 -1.784917e-07 -3.138192e-06 -5.045995e-07 -4.947792e-07 137.098837  75.858652 6.522383e-09
  5 137.098837  75.858652 2.842171e-14 0.000000e+00 1.037035e-14 3.980177e-15 137.098837  75.858652 7.564143e-17

Final T1, T2: 137.09883742088195 75.85865249812439
Final fluxes q1, q2, q3: 244.96073969103028 244.96073969103026 244.96073969103026


## Problem 12

In [87]:
# Initialization
nx = 4
ny = 4
T = np.zeros((nx, ny))

# Boundary conditions
T[0, :] = 25  # Top boundary
T[-1, :] = 75  # Bottom boundary
T[:, 0] = 200   # Left boundary
T[:, -1] = 0   # Right boundary

print("Initial temperature distribution with Boundary conditions:")
print(T)
print()

# Iterative solution
tolerance = 1e-4
max_iterations = 10000
for iteration in range(max_iterations):
    T_old = T.copy()
    for i in range(1, nx - 1):
        for j in range(1, ny - 1):
            T[i, j] = 0.25 * (T_old[i + 1, j] + T_old[i - 1, j] + T_old[i, j + 1] + T_old[i, j - 1])
    # Check for convergence
    if np.max(np.abs(T - T_old)) < tolerance:
        print(f"Converged after {iteration} iterations.")
        break
print()
print("Steady-state temperature distribution:")
print(T)


Initial temperature distribution with Boundary conditions:
[[200.  25.  25.   0.]
 [200.   0.   0.   0.]
 [200.   0.   0.   0.]
 [200.  75.  75.   0.]]

Converged after 19 iterations.

Steady-state temperature distribution:
[[200.          25.          25.           0.        ]
 [200.          93.74992847  43.74992847   0.        ]
 [200.         106.24992847  56.24992847   0.        ]
 [200.          75.          75.           0.        ]]
